# 0 Setting

In [242]:
# Parameter Setting
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
folder_name = 'Research-(D5) Synthesized input model'

config = {
    'learning_rate': 1e-5,
    'batch_size': 4,
    'seq_length': 5,

    'shuffle': False,
    'criterion': torch.nn.BCELoss(),
    'seed': 42,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'max_length': 512,
    'n_epochs': 3000,
    'early_stop': 50,
}

feature = [
    # X_1
    # 'input_ids',
    # 'attention_mask',
    # 'section_dummy',
    # 'publication_dummy',

    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    'today_return',
    # 'today_return_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',

    # 2. market index
    '^DJI',
    '^GSPC',
    '^NDX',
    '^IXIC',
    '^SOX',

    # y
    # '1_day_return',
    # '2_day_return',
    # '3_day_return',
    # '4_day_return',
    # '5_day_return',
    '1_day_return_cate',
    # '2_day_return_cate',
    # '3_day_return_cate',
    # '4_day_return_cate',
    # '5_day_return_cate',

    # Do not mark the datetime, it's for operation
    'datetime',
    ]

# All the news dataset
time_start = '2016-01-01T00:00:00'
time_end = '2020-04-02T00:00:00'

# time_start = '2016-01-01T00:00:00'
# time_end = '2018-12-31T00:00:00'

print(len(feature)-2)

stock_id_list = [
    'MSFT',
    "AMZN",
    "GOOG",
    "TSLA",
    "UBER",
    "JNJ",
    "BABA",
    "INTC",
    "IBM",
    "SONY",
    "ORCL",
    "PYPL",
    "CSCO",
    "ABNB",
    "NVDA",
    "QCOM",
    "CRM",
    "BIDU",
    "ADBE",
    "DELL",
    "HPQ",
    "MU",
    "AMD",
    "AVGO",
    "SAP",
    "TXN",
    "AMAT"
]

company_list = [
    'microsoft',
    "amazon",
    "google",
    "tesla",
    "uber",
    "johnson johnson",
    "alibaba",
    "intel",
    "ibm",
    "sony",
    "oracle",
    "paypal",
    "cisco",
    "airbnb",
    "nvidia",
    "qualcomm",
    "salesforce",
    "baidu",
    "adobe",
    "dell",
    "hp",
    "micron",
    "amd",
    "broadcom",
    "sap",
    "texas instruments",
    "applied material"
]

process_id = 1 #26

stock_id = stock_id_list[process_id]
company_name = company_list[process_id]
config_2 = {'input_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/data/2_'+company_name+'_for_model.csv',
            'save_path': '/content/drive/MyDrive/Colab Notebooks/'+folder_name+'/model_saved/model.ckpt',
            }

6


## (1) Import

In [243]:
# Google
from google.colab import drive
drive.mount('/content/drive')

# pip installation
!pip install transformers

# Basic
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# PyTorch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from transformers import XLNetModel, XLNetTokenizer, BertTokenizer, BertModel

# others
from datetime import datetime, timedelta
from tqdm import tqdm
from torchsummary import summary
import ast

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [244]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed for reproducibility
same_seed(config['seed'])


In [245]:
df = pd.read_csv(config_2['input_path'])
df = df.sort_values(by='datetime', ascending=True)
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,today_return,today_return_cate,1_day_return,...,datetime,^DJI,^GSPC,^NDX,^IXIC,^SOX,input_ids,attention_mask,section_dummy,publication_dummy
0,32.814499,32.886002,31.375500,31.849501,186290000,0.0,0.0,-0.029408,0,-0.005024,...,2016-01-04,-0.014739,-0.012531,0.002854,0.001111,0.008979,"[[101, 13423, 2023, 8983, 2006, 2115, 2849, 20...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
1,32.342999,32.345501,31.382500,31.689501,116452000,0.0,0.0,-0.020205,0,-0.001799,...,2016-01-05,0.000651,0.001455,-0.006175,-0.005370,-0.012003,"[[101, 16716, 11503, 999, 2793, 19093, 999, 21...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
2,31.100000,31.989500,31.015499,31.632500,106584000,0.0,0.0,0.017122,1,-0.039058,...,2016-01-06,-0.014475,-0.010663,0.007758,0.004570,-0.011092,"[[101, 8042, 2003, 1037, 2600, 2732, 1999, 214...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
3,31.090000,31.500000,30.260500,30.396999,141498000,0.0,0.0,-0.022290,0,-0.001464,...,2016-01-07,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667,"[[101, 1005, 9979, 1997, 1996, 16517, 1005, 20...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
4,30.983000,31.207001,30.299999,30.352501,110258000,0.0,0.0,-0.020350,0,0.017610,...,2016-01-08,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801,"[[101, 1520, 5392, 11579, 4548, 2026, 10007, 1...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,96.542999,96.989502,94.996002,95.004997,107758000,0.0,0.0,-0.015931,0,0.033604,...,2020-03-27,-0.011950,-0.005634,-0.009020,-0.006866,-0.015872,"[[101, 2129, 9733, 2038, 2506, 2000, 18077, 36...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, ..."
1066,96.141502,98.681503,95.616997,98.197502,122522000,0.0,0.0,0.021385,1,-0.007246,...,2020-03-30,0.029950,0.026444,0.027371,0.025146,0.025789,"[[101, 2235, 2449, 10940, 2071, 2468, 4539, 19...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, ..."
1067,98.217499,99.651001,97.200500,97.486000,102472000,0.0,0.0,-0.007448,0,-0.021552,...,2020-03-31,-0.013115,-0.011512,-0.005377,-0.005163,-0.020954,"[[101, 5278, 3268, 1024, 2057, 2024, 1006, 214...","[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,...","[[0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, ..."
1068,96.648499,97.248001,94.650002,95.385002,82438000,0.0,0.0,-0.013073,0,0.005834,...,2020-04-01,-0.013373,-0.011041,-0.012725,-0.013261,-0.018769,NaN,NaN,NaN,NaN


In [246]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits',
       'today_return', 'today_return_cate', '1_day_return', '2_day_return',
       '3_day_return', '4_day_return', '5_day_return', '1_day_return_cate',
       '2_day_return_cate', '3_day_return_cate', '4_day_return_cate',
       '5_day_return_cate', 'Sma', 'Rsi', 'Kd', 'Ema_12', 'Ema_26', 'Macd',
       'datetime', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX', 'input_ids',
       'attention_mask', 'section_dummy', 'publication_dummy'],
      dtype='object')

In [247]:
# Only contain selected features
df = df[feature]
df.columns

Index(['today_return', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX',
       '1_day_return_cate', 'datetime'],
      dtype='object')

## check data

In [248]:
df.isnull().sum()

today_return         0
^DJI                 0
^GSPC                0
^NDX                 0
^IXIC                0
^SOX                 0
1_day_return_cate    0
datetime             0
dtype: int64

In [249]:
df = df.dropna()
df = df.reset_index(drop=True)
df.isnull().sum()

today_return         0
^DJI                 0
^GSPC                0
^NDX                 0
^IXIC                0
^SOX                 0
1_day_return_cate    0
datetime             0
dtype: int64

## (2) Time Period Selection

In [250]:
# We use index to filter for time periods
df = df[(df['datetime']> time_start) & (df['datetime'] < time_end)]

# Drop datetime after using it
df.drop(columns=['datetime'], inplace=True)
df.shape

(1070, 7)

## (3) Transform str back to list

In [251]:
df.isnull().sum()

today_return         0
^DJI                 0
^GSPC                0
^NDX                 0
^IXIC                0
^SOX                 0
1_day_return_cate    0
dtype: int64

In [252]:
# # 将字符串转换回列表的函数
# def string_to_list(s):
#     return ast.literal_eval(s)

# # 将列中的字符串转换回列表
# df['input_ids'] = df['input_ids'].apply(string_to_list)
# df['attention_mask'] = df['attention_mask'].apply(string_to_list)
# df['section_dummy'] = df['section_dummy'].apply(string_to_list)
# df['publication_dummy'] = df['publication_dummy'].apply(string_to_list)

## (4) Train_test_split

In [253]:
# 1. Set up X, y
to_remove_list = ['datetime', '1_day_return_cate']

# Filter out values in to_remove_list
filtered_list = [x for x in feature if x not in to_remove_list]

X = df[filtered_list]
y = df['1_day_return_cate']

In [254]:
# print(X['section_dummy'])
# print(type(X['section_dummy'][0]))
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)

# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)\
# temp_array = np.array(X['section_dummy'][0])
# print(temp_array.dtype)


In [255]:
# def int_to_float(int_list):
#   float_list = np.array(int_list, dtype=np.float32)
#   return float_list
# X['section_dummy'] = X['section_dummy'].apply(int_to_float)
# X['publication_dummy'] = X['publication_dummy'].apply(int_to_float)

In [256]:
# Check X, y shape
print('X:', X.shape)
print('y:', y.shape)

X: (1070, 6)
y: (1070,)


In [257]:
# 2. train_test_split
# val dataset for final examination

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=config['test_ratio'], random_state=config['seed'], shuffle=config['shuffle'])
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=config['valid_ratio'], random_state=config['seed'], shuffle=config['shuffle'])
X_train


,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,-0.029408,-0.014739,-0.012531,0.002854,0.001111,0.008979
1,-0.020205,0.000651,0.001455,-0.006175,-0.005370,-0.012003
2,0.017122,-0.014475,-0.010663,0.007758,0.004570,-0.011092
3,-0.022290,-0.022161,-0.021271,-0.009984,-0.009917,-0.010667
4,-0.020350,-0.010456,-0.012302,-0.015069,-0.016601,-0.019801
...,...,...,...,...,...,...
679,-0.005065,0.002379,0.002530,0.003512,0.001722,-0.001817
680,-0.011410,-0.000569,-0.000482,-0.003944,-0.002008,0.005629
681,-0.023891,-0.003424,-0.005176,-0.011679,-0.012064,-0.009262
682,0.011675,0.006548,0.004694,0.007195,0.006648,0.003821


## (4.5) Resampling

In [258]:
# from sklearn.utils import resample

# # Train set
# train_df = pd.concat([X_train, y_train], axis=1)

# def undersample_majority_class(df, class_column):
#     minority_class = df[class_column].value_counts().idxmin()
#     majority_class = df[class_column].value_counts().idxmax()

#     minority_samples = df[df[class_column] == minority_class]
#     majority_samples = df[df[class_column] == majority_class]

#     majority_samples_undersampled = resample(majority_samples, replace=False, n_samples=len(minority_samples))
#     undersampled_df = pd.concat([minority_samples, majorit y_samples_undersampled])

#     return undersampled_df

# target_name = 't+1_day_trend_cate'
# # 下採樣多數類別
# undersampled_train_df = undersample_majority_class(train_df, target_name)

# # 分割回 X_train 和 y_train
# X_train = undersampled_train_df.drop(target_name, axis=1)
# y_train = undersampled_train_df[target_name]

# # 打印下採樣後的類別數量
# print("Undersampled Train Class Counts:\n", y_train.value_counts())

# # Val Set
# val_df = pd.concat([X_val, y_val], axis=1)
# undersampled_val_df = undersample_majority_class(val_df, target_name)
# X_val = undersampled_val_df.drop(target_name, axis=1)
# y_val = undersampled_val_df[target_name]

# # Test Set
# test_df = pd.concat([X_test, y_test], axis=1)
# undersampled_test_df = undersample_majority_class(test_df, target_name)
# X_test = undersampled_test_df.drop(target_name, axis=1)
# y_test = undersampled_test_df[target_name]

## (5) Scaler

In [259]:
scale_feature = [
    # X_2
    # 1. tech indicator
    # 'Open',
    # 'High',
    # 'Low',
    # 'Close',
    # 'Volume',
    # 'Dividends',
    # 'Stock Splits',
    # 'Today_trend',
    # 'Today_trend_cate',
    # 'Sma',
    # 'Rsi',
    # 'Kd',
    # 'Ema_12',
    # 'Ema_26',
    # 'Macd',

    # 2. market index
    # '^DJI',
    '^GSPC',
    # '^NDX',
    # '^IXIC',
    # '^SOX',
    # 'datetime'
    ]

def CustomScaler(X_train, X_val, X_test):
  scaler = MinMaxScaler()
  for i in scale_feature:

    # 對特定欄位進行標準化
    X_train_scaled = scaler.fit_transform(X_train[[i]])
    X_val_scaled = scaler.transform(X_val[[i]])
    X_test_scaled = scaler.transform(X_test[[i]])

    # 將標準化後的值重新賦值給 DataFrame
    X_train[i] = X_train_scaled
    X_val[i] = X_val_scaled
    X_test[i] = X_test_scaled

  return X_train, X_val, X_test

X_train, X_val, X_test = CustomScaler(X_train, X_val, X_test)

X_train

,today_return,^DJI,^GSPC,^NDX,^IXIC,^SOX
0,-0.029408,-0.014739,0.377236,0.002854,0.001111,0.008979
1,-0.020205,0.000651,0.578553,-0.006175,-0.005370,-0.012003
2,0.017122,-0.014475,0.404126,0.007758,0.004570,-0.011092
3,-0.022290,-0.022161,0.251419,-0.009984,-0.009917,-0.010667
4,-0.020350,-0.010456,0.380522,-0.015069,-0.016601,-0.019801
...,...,...,...,...,...,...
679,-0.005065,0.002379,0.594032,0.003512,0.001722,-0.001817
680,-0.011410,-0.000569,0.550677,-0.003944,-0.002008,0.005629
681,-0.023891,-0.003424,0.483104,-0.011679,-0.012064,-0.009262
682,0.011675,0.006548,0.625183,0.007195,0.006648,0.003821


## (6) Check number

In [260]:
def calculate_class_stats(y):
    class_counts = y.value_counts()
    total_samples = len(y)
    class_ratios = class_counts / total_samples
    return class_counts, class_ratios

# 計算類別數量和比例
train_class_counts, train_class_ratios = calculate_class_stats(y_train)
val_class_counts, val_class_ratios = calculate_class_stats(y_val)
test_class_counts, test_class_ratios = calculate_class_stats(y_test)

# 創建包含數量和比例的 DataFrame
class_stats = pd.DataFrame({
    'Train Count': train_class_counts,
    'Train Ratio': train_class_ratios,
    'Validation Count': val_class_counts,
    'Validation Ratio': val_class_ratios,
    'Test Count': test_class_counts,
    'Test Ratio': test_class_ratios
})

# 打印 DataFrame
print(class_stats)


   Train Count  Train Ratio  Validation Count  Validation Ratio  Test Count  \
1          394     0.576023                88          0.511628         111   
0          290     0.423977                84          0.488372         103   

   Test Ratio  
1    0.518692  
0    0.481308  


In [261]:
# Time period
print('Time Period')
print('From:', time_start)
print('To:', time_end, '\n')

# Sample size
print('Sample size:', X.shape[0])
print('Feature:', X.columns, '\n')
print('Target:', y.name, '\n')
print('Train: Val: Test = 0.81: 0.1: 0.09=', X_train.shape[0], X_test.shape[0], X_val.shape[0])

Time Period
From: 2016-01-01T00:00:00
To: 2020-04-02T00:00:00 

Sample size: 1070
Feature: Index(['today_return', '^DJI', '^GSPC', '^NDX', '^IXIC', '^SOX'], dtype='object') 

Target: 1_day_return_cate 

Train: Val: Test = 0.81: 0.1: 0.09= 684 214 172


# Model

## (1) Dataset & Dataloader

In [262]:
# Dataset
# X_1 =['input_ids', 'attention_mask', 'section_dummy', 'publication_dummy']


class CustomDataset(Dataset):
    def __init__(self, X, y, config):
        # X_2
        self.X_2 = torch.tensor(X.values, dtype=torch.float)

        # y
        self.y = torch.tensor(y.values, dtype=torch.float)

        # other setting
        self.len = X.shape[0]
        self.seq_length = config['seq_length']

    def __getitem__(self,idx):
        # X_2
        X_2 = self.X_2[idx : idx + self.seq_length]

        # 3. y
        y = self.y[idx + self.seq_length - 1]

        return X_2, y

    def __len__(self):
        return self.len - self.seq_length

In [263]:
# DataLoader
train_dataset = CustomDataset(X_train, y_train, config)
val_dataset = CustomDataset(X_val, y_val, config)
test_dataset = CustomDataset(X_test, y_test, config)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=config['shuffle'], pin_memory=True)

In [264]:
# Check loader output
for batch in train_loader:
    X_2, y = batch

    # 打印批次数据的形状，以确保它们符合预期
    # print("Input IDs shape:", input_ids.shape)
    # print("Attention Mask shape:", attention_mask.shape)
    # print("Section shape:", section.shape)
    # print("Publication shape:", publication.shape)
    print("X_2 shape:", X_2.shape)
    print("Labels shape:", y.shape)

    # print("Input IDs:", input_ids)
    # print("Attention Mask:", attention_mask)
    # print("Section:", section)
    # print("Publication:", publication)
    print("X_2:", X_2)
    print("Labels:", y)

    break


X_2 shape: torch.Size([4, 5, 6])
Labels shape: torch.Size([4])
X_2: tensor([[[-0.0294, -0.0147,  0.3772,  0.0029,  0.0011,  0.0090],
         [-0.0202,  0.0007,  0.5786, -0.0062, -0.0054, -0.0120],
         [ 0.0171, -0.0145,  0.4041,  0.0078,  0.0046, -0.0111],
         [-0.0223, -0.0222,  0.2514, -0.0100, -0.0099, -0.0107],
         [-0.0203, -0.0105,  0.3805, -0.0151, -0.0166, -0.0198]],

        [[-0.0202,  0.0007,  0.5786, -0.0062, -0.0054, -0.0120],
         [ 0.0171, -0.0145,  0.4041,  0.0078,  0.0046, -0.0111],
         [-0.0223, -0.0222,  0.2514, -0.0100, -0.0099, -0.0107],
         [-0.0203, -0.0105,  0.3805, -0.0151, -0.0166, -0.0198],
         [ 0.0086,  0.0024,  0.5393, -0.0041, -0.0076, -0.0045]],

        [[ 0.0171, -0.0145,  0.4041,  0.0078,  0.0046, -0.0111],
         [-0.0223, -0.0222,  0.2514, -0.0100, -0.0099, -0.0107],
         [-0.0203, -0.0105,  0.3805, -0.0151, -0.0166, -0.0198],
         [ 0.0086,  0.0024,  0.5393, -0.0041, -0.0076, -0.0045],
         [-0.0118,

## (2) Model Architecture

In [265]:
class MyModel(nn.Module):
    def __init__(self, config):
        super(MyModel, self).__init__()
        self.seq_length = config['seq_length']
        self.lstm_1 = nn.LSTM(6, 32, num_layers=3,dropout=0.2, batch_first=True, bidirectional=False)
        # self.lstm_2 = nn.LSTM(32, 64, num_layers=1, batch_first=True, bidirectional=False)
        # self.lstm_3 = nn.LSTM(64, 16, num_layers=1, batch_first=True, bidirectional=False)
        # self.dropout = nn.Dropout(0.2)
        self.sequential = nn.Sequential(
            nn.Linear(32, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, X_2):
        h_tech_list = []
        # for i in range(self.seq_length):
        #   out = X_2[:, i, :]
        #   h_tech_list.append(out)
        # h_tech = torch.stack(h_tech_list, dim=1)  # 5 [64, 500] tensor to 1 [64, 5, 500] tensor
        h_tech = X_2


        # 3. LSTM
        out, _ = self.lstm_1(h_tech)
        # out = self.dropout(out)

        # out, _ = self.lstm_2(out)
        # out = self.dropout(out)

        # out, _ = self.lstm_3(out)
        # out = self.dropout(out)
        out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term

        out = self.sequential(out)
        final_out = self.sigmoid(out)

        return final_out


## (3) Trainer

### 1 BCELoss

In [266]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']
#     optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#       for X_2, y in train_loader:
#         optimizer.zero_grad()  # Set gradient to zero

#         # Forward pass
#         X_2, y = X_2.to(device), y.to(device)
#         pred = model(X_2)

#         y = y.reshape(-1, 1)
#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       # 2. Evaluation
#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       for X_2, y in val_loader:
#           X_2, y = X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(X_2)
#               y = y.reshape(-1, 1)
#               loss = criterion(pred, y)
#           loss_record.append(loss.item())

#       # Mean
#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       # 3. Judge of saving model
#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


2. Can see valid acc

In [267]:
def trainer(model, train_loader, val_loader, config, device):

    criterion = config['criterion']
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

    writer = SummaryWriter()  # Writer of tensoboard.
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    # 1. Training
    for epoch in range(n_epochs):
      model.train()  # Set the model to training mode
      loss_record = []

      train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
      for X_2, y in train_loader:
        optimizer.zero_grad()  # Set gradient to zero

        # Forward pass
        X_2, y = X_2.to(device), y.to(device)
        pred = model(X_2)

        y = y.reshape(-1, 1)
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

      mean_train_loss = sum(loss_record)/len(loss_record)
      writer.add_scalar('Loss/train', mean_train_loss, step)

      # 2. Evaluation
      model.eval() # Set your model to evaluation mode.
      loss_record = []
      correct = 0
      total = 0
      predicted_labels_list = []
      targets_list = []
      for X_2, y in val_loader:
          X_2, y = X_2.to(device), y.to(device)
          with torch.no_grad():
              pred = model(X_2)
              y = y.reshape(-1, 1)
              loss = criterion(pred, y)
              # _, predicted = torch.max(pred, 1)
              predicted_labels = (pred >= 0.5).float()
              predicted_labels_list.extend(predicted_labels.tolist())
              targets_list.extend(y.tolist())

          loss_record.append(loss.item())
      accuracy = accuracy_score(targets_list, predicted_labels_list)

      # Mean
      mean_valid_loss = sum(loss_record)/len(loss_record)
      print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}, Val Acc: {accuracy:.4f}')
      writer.add_scalar('Loss/valid', mean_valid_loss, step)

      # 3. Judge of saving model
      if mean_valid_loss < best_loss:
          best_loss = mean_valid_loss
          torch.save(model.state_dict(), config_2['save_path']) # Save your best model
          print('Saving model with loss {:.3f}...'.format(best_loss))
          early_stop_count = 0
      else:
          early_stop_count += 1

      if early_stop_count >= config['early_stop']:
          print('\nModel is not improving, so we halt the training session.')
          return


Epoch [20/3000]:   0%|          | 0/170 [01:38<?, ?it/s, loss=0.625]


2. Accuracy-Based Trainer

In [268]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']
#     optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_accuracy, step, early_stop_count = config['n_epochs'], 0.0, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#         model.train()  # Set the model to training mode
#         loss_record = []

#         train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#         for X_2, y in train_loader:
#             optimizer.zero_grad()  # Set gradient to zero

#             # Forward pass
#             X_2, y = X_2.to(device), y.to(device)
#             pred = model(X_2)

#             y = y.reshape(-1, 1)
#             loss = criterion(pred, y)
#             loss.backward()                     # Compute gradient(backpropagation).
#             optimizer.step()                    # Update parameters.
#             step += 1
#             loss_record.append(loss.detach().item())

#             train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]') # Display current epoch number and loss on tqdm progress bar.
#             train_pbar.set_postfix({'loss': loss.detach().item()})

#         mean_train_loss = sum(loss_record)/len(loss_record)
#         writer.add_scalar('Loss/train', mean_train_loss, step)

#         # 2. Evaluation
#         model.eval() # Set your model to evaluation mode.
#         correct = 0
#         total = 0

#         for X_2, y in val_loader:
#             X_2, y = X_2.to(device), y.to(device)
#             with torch.no_grad():
#                 pred = model(X_2)
#                 y = y.reshape(-1, 1)
#                 loss = criterion(pred, y)
#                 _, predicted = torch.max(pred, 1)
#                 total += y.size(0)
#                 correct += (predicted == y).sum().item()

#         accuracy = correct / total
#         accuracy = accuracy/config['batch_size']

#         # Print and log accuracy
#         print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Validation Accuracy: {accuracy:.4f}')
#         writer.add_scalar('Accuracy/valid', accuracy, step)

#         # 3. Judge of saving model based on accuracy
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#             print('Saving model with accuracy {:.3f}...'.format(best_accuracy))
#             early_stop_count = 0
#         else:
#             early_stop_count += 1

#         if early_stop_count >= config['early_stop']:
#             print('\nModel is not improving, so we halt the training session.')
#             return


### 2 Balanced Cross Entropy

In [269]:
print(train_class_counts)

print(train_class_counts[0])

1    394
0    290
Name: 1_day_return_cate, dtype: int64
290


In [270]:
X_train.shape[0]

684

In [271]:
# # # 2. training set-based

# # # smooth factor
# # smooth = 1e-15

# # total_samples = X_train.shape[0]
# # pos_weight = (total_samples) / (train_class_counts[1]*2 + smooth)
# # neg_weight = (total_samples) / (train_class_counts[0]*2 + smooth)

# # Beta is usealess, Alpha is for adjusting Balance power.
# beta = 2
# alpha = 1.28 # 1.25 < alpha < 1.27
# class BalancedBCELoss(nn.Module):
#     def __init__(self, beta):
#         super(BalancedBCELoss, self).__init__()

#     def forward(self, input, target):
#         # Number counts
#         num_pos = torch.sum(target == 1).float()
#         num_neg = torch.sum(target == 0).float()

#         # smooth factor
#         smooth = 1e-15

#         # Weight Calculation
#         # 1. Batch-based
#         total_samples = num_pos + num_neg
#         pos_weight = (total_samples) / (num_pos*beta + smooth) * alpha
#         neg_weight = (total_samples) / (num_neg*beta + smooth) * (1/alpha)

#         pos_weight = pos_weight

#         # 根据目标张量的值创建判断式
#         weights = torch.where(target == 1, torch.tensor(pos_weight), torch.tensor(neg_weight))

#         # 使用nn.BCEWithLogitsLoss，并应用平衡权重
#         criterion = nn.BCEWithLogitsLoss(weight=weights)

#         return criterion(input, target)

In [272]:
# def trainer(model, train_loader, val_loader, config, device):

#     criterion = BalancedBCELoss(beta=beta)
#     optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

#     writer = SummaryWriter()  # Writer of tensoboard.
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     # 1. Training
#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       train_pbar = tqdm(train_loader, position=0, leave=True)  # tqdm is a package to visualize your training progress.
#       for input_ids, attention_mask, section, publication, X_2, y in train_loader:
#         optimizer.zero_grad()  # Set gradient to zero

#         # Forward pass
#         input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#         pred = model(input_ids, attention_mask, section, publication, X_2)

#         y = y.reshape(-1, 1)
#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       # 2. Evaluation
#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       for input_ids, attention_mask, section, publication, X_2, y in val_loader:
#           input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(input_ids, attention_mask, section, publication, X_2)
#               y = y.reshape(-1, 1)
#               loss = criterion(pred, y)
#           loss_record.append(loss.item())

#       # Mean
#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       # 3. Judge of saving model
#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


### 3 Training by parts (分段訓練)

In [273]:
# # 嘗試分段訓練

# def trainer1(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']

#     # Stage 1: Train base_model and fc1
#     optimizer_stage1 = torch.optim.Adam([
#         {'params': model.base_model.parameters(), 'lr': config['learning_rate']},
#         {'params': model.fc1.parameters(), 'lr': config['learning_rate']}
#     ])

#     # Writer of tensoboard.
#     writer = SummaryWriter()

#     # Not sure what's these for
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       # tqdm is a package to visualize your training progress.
#       train_pbar = tqdm(train_loader, position=0, leave=True)
#       for input_ids_tensor, attention_mask_tensor, X_2, y in train_loader:
#         # Set gradient to zero
#         optimizer_stage1.zero_grad()

#         # Forward pass
#         input_ids_tensor, attention_mask_tensor, X_2, y = input_ids_tensor.to(device), attention_mask_tensor.to(device), X_2.to(device), y.to(device)
#         pred = model(input_ids_tensor, attention_mask_tensor, X_2)

#         y = y.reshape(-1, 1)

#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer_stage1.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       for input_ids_tensor, attention_mask_tensor, X_2, y in val_loader:
#           input_ids_tensor, attention_mask_tensor, X_2, y = input_ids_tensor.to(device), attention_mask_tensor.to(device), X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(input_ids_tensor, attention_mask_tensor, X_2)
#               y = y.reshape(-1, 1)
#               loss = criterion(pred, y)

#           loss_record.append(loss.item())

#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


# def trainer2(model, train_loader, val_loader, config, device):

#     criterion = config['criterion']

#     # Stage 2: Train lstm1 and fc2
#     optimizer_stage2 = torch.optim.Adam([
#         {'params': model.lstm1.parameters(), 'lr': config['learning_rate']},
#         {'params': model.fc2.parameters(), 'lr': config['learning_rate']}
#     ])

#     # Writer of tensoboard.
#     writer = SummaryWriter()

#     # Not sure what's these for
#     n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

#     for epoch in range(n_epochs):
#       model.train()  # Set the model to training mode
#       loss_record = []

#       # tqdm is a package to visualize your training progress.
#       train_pbar = tqdm(train_loader, position=0, leave=True)
#       for input_ids_tensor, attention_mask_tensor, X_2, y in train_loader:
#         # Set gradient to zero
#         optimizer_stage2.zero_grad()

#         # Forward pass
#         input_ids_tensor, attention_mask_tensor, X_2, y = input_ids_tensor.to(device), attention_mask_tensor.to(device), X_2.to(device), y.to(device)
#         pred = model(input_ids_tensor, attention_mask_tensor, X_2)

#         y = y.reshape(-1, 1)

#         loss = criterion(pred, y)
#         loss.backward()                     # Compute gradient(backpropagation).
#         optimizer_stage2.step()                    # Update parameters.
#         step += 1
#         loss_record.append(loss.detach().item())

#         # Display current epoch number and loss on tqdm progress bar.
#         train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
#         train_pbar.set_postfix({'loss': loss.detach().item()})

#       mean_train_loss = sum(loss_record)/len(loss_record)
#       writer.add_scalar('Loss/train', mean_train_loss, step)

#       model.eval() # Set your model to evaluation mode.
#       loss_record = []
#       for input_ids_tensor, attention_mask_tensor, X_2, y in val_loader:
#           input_ids_tensor, attention_mask_tensor, X_2, y = input_ids_tensor.to(device), attention_mask_tensor.to(device), X_2.to(device), y.to(device)
#           with torch.no_grad():
#               pred = model(input_ids_tensor, attention_mask_tensor, X_2)
#               y = y.reshape(-1, 1)
#               loss = criterion(pred, y)

#           loss_record.append(loss.item())

#       mean_valid_loss = sum(loss_record)/len(loss_record)
#       print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
#       writer.add_scalar('Loss/valid', mean_valid_loss, step)

#       if mean_valid_loss < best_loss:
#           best_loss = mean_valid_loss
#           torch.save(model.state_dict(), config_2['save_path']) # Save your best model
#           print('Saving model with loss {:.3f}...'.format(best_loss))
#           early_stop_count = 0
#       else:
#           early_stop_count += 1

#       if early_stop_count >= config['early_stop']:
#           print('\nModel is not improving, so we halt the training session.')
#           return


## (4) Load Model

1. Load pretrain model

In [274]:
# 載入預訓練模型
# base_model = BertModel.from_pretrained('bert-base-uncased')
# base_model = PreModel(base_model)
# base_model.load_state_dict(torch.load(config_2['pretrained_model_path']))

# Parameter
# section_length = len(df['section_dummy'][0])
# publication_length = len(df['publication_dummy'][0])
# X_2_length = len(feature) - 6

2-1. Initial Model

In [275]:
model = MyModel(config)
model.to(device)

MyModel(
  (lstm_1): LSTM(6, 32, num_layers=3, batch_first=True, dropout=0.2)
  (sequential): Sequential(
    (0): Linear(in_features=32, out_features=1, bias=True)
  )
  (sigmoid): Sigmoid()
)

2-2. Contunue training from model.ckpt

In [276]:
# model = MyModel(base_model, config, section_length, publication_length, X_2_length)
# model.load_state_dict(torch.load(config_2['continue_model_path']))
# model.to(device)

## (5) Require_grad

In [277]:

# # Freeze all layers
# for param in model.pretrained_model.parameters():
#   param.requires_grad = False

# for param in model.pretrained_model.base_model.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.pretrained_model.fc1.parameters():
#     param.requires_grad = True

# # for param in model.fc1.parameters():
# #     param.requires_grad = True

# # Check requires_grad status
# for name, param in model.named_parameters():
#     print(name, param.requires_grad)

In [278]:
# Freeze all layers
# for param in model.base_model.parameters():
#   param.requires_grad = False

# Unfreeze part of layers
# for param in model.base_model.encoder.layer[6].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[7].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[8].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[9].parameters():
#     param.requires_grad = True

# for param in model.base_model.encoder.layer[10].parameters():
#     param.requires_grad = True

# for param in model.base_model.base_model.encoder.layer[11].parameters():
#     param.requires_grad = True

# for param in model.base_model.fc1.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc2.parameters():
#     param.requires_grad = True

# for param in model.base_model.fc3.parameters():
    # param.requires_grad = True

# Check requires_grad status
for name, param in model.named_parameters():
    print(name, param.requires_grad)

lstm_1.weight_ih_l0 True
lstm_1.weight_hh_l0 True
lstm_1.bias_ih_l0 True
lstm_1.bias_hh_l0 True
lstm_1.weight_ih_l1 True
lstm_1.weight_hh_l1 True
lstm_1.bias_ih_l1 True
lstm_1.bias_hh_l1 True
lstm_1.weight_ih_l2 True
lstm_1.weight_hh_l2 True
lstm_1.bias_ih_l2 True
lstm_1.bias_hh_l2 True
sequential.0.weight True
sequential.0.bias True


# Training

In [279]:
# 全部訓練
trainer(model, train_loader, val_loader, config, device)

# # 分段訓練
# trainer2(model, train_loader, val_loader, config, device)
# trainer1(model, train_loader, val_loader, config, device)

Epoch [1/3000]:   0%|          | 0/170 [00:01<?, ?it/s, loss=0.672]

Epoch [1/3000]: Train loss: 0.7065, Valid loss: 0.6967, Val Acc: 0.4910
Saving model with loss 0.697...


Epoch [2/3000]:   0%|          | 0/170 [00:01<?, ?it/s, loss=0.675]

Epoch [2/3000]: Train loss: 0.7053, Valid loss: 0.6963, Val Acc: 0.4910
Saving model with loss 0.696...


Epoch [3/3000]:   0%|          | 0/170 [00:01<?, ?it/s, loss=0.677]

...

Epoch [32/3000]: Train loss: 0.6840, Valid loss: 0.6951, Val Acc: 0.5090


Epoch [33/3000]:   0%|          | 0/170 [00:01<?, ?it/s, loss=0.735]

Epoch [33/3000]: Train loss: 0.6831, Valid loss: 0.6954, Val Acc: 0.5090


Epoch [34/3000]:   0%|          | 0/170 [00:00<?, ?it/s, loss=0.611]

KeyboardInterrupt: ignored

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# Evaluate

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)

# Evaluation mode
model.eval()
accuracy_list = []
threshold = 0.5

# 1. Train part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_train_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= threshold).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# 2. Val part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_val_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= 0.5).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))


# 3. Test part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_test_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= 0.5).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# print(model, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)

In [ ]:
# Evaluation Dataloader
con_train_loader = DataLoader(train_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_val_loader = DataLoader(val_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)
con_test_loader = DataLoader(test_dataset, batch_size=1, shuffle=config['shuffle'], pin_memory=True)

model = MyModel(config)
model.load_state_dict(torch.load(config_2['save_path']))
model.to(device)

# Evaluation mode
model.eval()
accuracy_list = []
threshold = 0.51

# 1. Train part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_train_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= threshold).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================================================================================')
print('Training Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list), '\n')

# 2. Val part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_val_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= threshold).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================')
print('Val Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))


# 3. Test part
predicted_labels_list = []
targets_list = []

with torch.no_grad():
    for X_2, y in con_test_loader:
        X_2, y = X_2.to(device), y.to(device)
        outputs = model(X_2)
        # _, predicted_labels = torch.max(outputs, dim=1)  # 获取每个样本预测的类别索引
        predicted_labels = (outputs >= threshold).float()

        predicted_labels_list.extend(predicted_labels.tolist())
        targets_list.extend(y.tolist())

accuracy = accuracy_score(targets_list, predicted_labels_list)

print('=====================================================')
print('Testing Result:')
print(classification_report(targets_list, predicted_labels_list))
print(confusion_matrix(targets_list, predicted_labels_list))

print('=====================================================', '\n')
print("Accuracy [Train, Val, Test]: ", accuracy_list, '\n')
# print(model, '\n')
print('Config: ', config, '\n')
print('Feature: ', feature)
print('time_start: ', time_start, 'time_end: ', time_end)

## (2) Thresholds

In [ ]:
# # function of evaluation
# def evaluate_model(model, dataloader, threshold=0.5, device='cuda'):
#     predicted_labels_list = []
#     targets_list = []

#     with torch.no_grad():
#         for input_ids, attention_mask, section, publication, X_2, y in dataloader:
#             input_ids, attention_mask, section, publication, X_2, y = input_ids.to(device), attention_mask.to(device), section.to(device), publication.to(device), X_2.to(device), y.to(device)
#             outputs = model(input_ids, attention_mask, section, publication, X_2)
#             predicted_labels = (outputs >= threshold).float()

#             predicted_labels_list.extend(predicted_labels.tolist())
#             targets_list.extend(y.tolist())

#     accuracy = accuracy_score(targets_list, predicted_labels_list)

#     print('=====================================================================================================================')
#     print(str(dataloader), ' Result:')
#     print(classification_report(targets_list, predicted_labels_list))
#     print(confusion_matrix(targets_list, predicted_labels_list), '\n')

In [ ]:
# accuracy_list = []
# threshold = 0.51

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.52

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.53

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.54

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.55

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.56

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.57

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

In [ ]:
# accuracy_list = []
# threshold = 0.58

# evaluate_model(model, con_train_loader, threshold=threshold)
# evaluate_model(model, con_val_loader, threshold=threshold)
# evaluate_model(model, con_test_loader, threshold=threshold)

# Benchmark

In [ ]:
# class BenchmarkModel(nn.Module):
#     def __init__(self, config, section_length, publication_length, X_2_length):
#         super(BenchmarkModel, self).__init__()
#         # tech
#         self.fc_h_tech = nn.Linear(X_2_length, config['h_tech_size'])

#         # 3. LSTM
#         self.lstm_1 = nn.LSTM(config['h_size'], 2, num_layers=1, batch_first=True, bidirectional=False)
#         self.sequential = nn.Sequential(
#             nn.Linear(2, 1)
#         )
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, input_ids, attention_mask, section, publication, X_2):
#         # h_tech
#         h_tech = self.fc_h_tech(X_2)

#         # 3. LSTM
#         out, _ = self.lstm_1(h_tech)
#         out = out[:, -1, :]  # Get the last one of LSTM output for prediction of next-term
#         # print('out 1(last layer of LSTM): ', out.shape, out)
#         out = self.sequential(out)
#         # print('out 2(after sequnetial): ',out.shape , out)
#         final_out = self.sigmoid(out)
#         # print('final_out: ', final_out.shape, final_out)

#         return final_out
